In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

train_data = pd.read_csv('linear_regression_train(2).txt', delimiter=r"\s+|,", engine='python', header=None)
test_data = pd.read_csv('linear_regression_test(2).txt', delimiter=r"\s+|,", engine='python', header=None)

In [2]:
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
X_test = test_data.values

## 2.1.1

In [3]:
# 初始化和训练模型
model = LinearRegression()
model.fit(X_train, y_train)

# 提取模型的偏置 (w0) 和系数 (w1, ..., w11)
w0 = model.intercept_
coefficients = model.coef_

print(f"Bias (w0): {w0}")
for idx, coef in enumerate(coefficients, start=1):
    print(f"w{idx}: {coef}")

# 在测试集上进行预测
y_pred = model.predict(X_test)

# 打印测试集的预测结果
print("\nPredicted y values for the test set:")
for i, pred in enumerate(y_pred, start=1):
    print(f"Test Sample {i}: {pred}")


Bias (w0): 3.613646026669057
w1: 0.015325690386245282
w2: 0.00025227640634062354
w3: 0.0007203868499650188
w4: 0.9991635645789793
w5: 0.9997402360310111
w6: 1.0006233993583653
w7: 0.9988323588429907
w8: 1.0000013015236566
w9: 1.0002245483558714
w10: 0.9990396151482023
w11: 0.9993448460859428

Predicted y values for the test set:
Test Sample 1: -56.11129687958479
Test Sample 2: -173.51651970931792
Test Sample 3: -6.770877912227151
Test Sample 4: 209.517090441872
Test Sample 5: 116.89029785098526
Test Sample 6: -100.29084527235767
Test Sample 7: -310.12783900144876
Test Sample 8: 501.3863019426081
Test Sample 9: 244.1147678085642
Test Sample 10: 18.566393254550192


## 2.1.2

In [4]:
# 在 X_train 中添加一列全为 1 的列，作为偏置项
X_train_b = np.hstack((np.ones((X_train.shape[0], 1)), X_train))  # 维度: (2024, 12)
X_test_b = np.hstack((np.ones((X_test.shape[0], 1)), X_test))    # 维度: (10, 12)

# 根据正则方程计算系数 w_hat
XtX = np.dot(X_train_b.T, X_train_b)
XtX_inv = np.linalg.inv(XtX)  # 求逆矩阵
Xt_y = np.dot(X_train_b.T, y_train)
w_hat = np.dot(XtX_inv, Xt_y)  # 维度: (12, )

# 提取偏置和系数
w0_manual = w_hat[0]
coefficients_manual = w_hat[1:]

print(f"Bias (w0): {w0_manual}")
for idx, coef in enumerate(coefficients_manual, start=1):
    print(f"w{idx}: {coef}")

# 使用手动计算的模型进行预测
y_pred_manual = np.dot(X_test_b, w_hat)

# 打印预测结果
print("\nPredicted y values for the test set (manual calculation):")
for i, pred in enumerate(y_pred_manual, start=1):
    print(f"Test Sample {i}: {pred}")


Bias (w0): 3.6136460266688557
w1: 0.015325690386246638
w2: 0.0002522764063409254
w3: 0.0007203868499652336
w4: 0.9991635645789788
w5: 0.9997402360310117
w6: 1.000623399358362
w7: 0.99883235884299
w8: 1.0000013015236562
w9: 1.0002245483558732
w10: 0.9990396151482013
w11: 0.9993448460859436

Predicted y values for the test set (manual calculation):
Test Sample 1: -56.11129687958453
Test Sample 2: -173.5165197093183
Test Sample 3: -6.770877912227066
Test Sample 4: 209.51709044187174
Test Sample 5: 116.89029785098508
Test Sample 6: -100.29084527235786
Test Sample 7: -310.1278390014489
Test Sample 8: 501.38630194260753
Test Sample 9: 244.11476780856378
Test Sample 10: 18.566393254550302


## 2.2.1

In [ ]:
import pandas as pd
import numpy as np
from math import floor
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

# 读取Excel文件
df = pd.read_excel('Classification iris(2).xlsx')

# 初始化训练集和测试集的列表
train_ids = []
test_ids = []

# 按类别分组
grouped = df.groupby('class')

for class_name, group in grouped:
    n_samples = len(group)
    n_train = floor(0.7 * n_samples)
    
    # 获取前70%的实例ID作为训练集
    train_ids.extend(group.iloc[:n_train]['instance_id'].tolist())
    
    # 获取剩余30%的实例ID作为测试集
    test_ids.extend(group.iloc[n_train:]['instance_id'].tolist())

print("Q2.2.1  Split training set and test set:")
print(f"Training set: {train_ids}")
print(f"Test set: {test_ids}")

In [ ]:
# 分别获取训练集和测试集的数据
train_df = df[df['instance_id'].isin(train_ids)].reset_index(drop=True)
test_df = df[df['instance_id'].isin(test_ids)].reset_index(drop=True)

# 特征和标签
X_train = train_df[['sepal length', 'sepal width', 'petal length', 'petal width']].values  # 105x4
y_train = train_df['class'].values  # 105x1

X_test = test_df[['sepal length', 'sepal width', 'petal length', 'petal width']].values  # 45x4
y_test = test_df['class'].values    # 45x1

# 标签编码（将类别标签转换为数值）
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# 获取所有类别
classes = le.classes_

# 生成字典存储每个class的模型、weight、b、支持向量索引
classifiers = {}
w_dict = {}
b_dict = {}
support_vectors_dict = {}

# 训练每个类别的SVM模型
for idx, cls in enumerate(classes):

    # 创建二分类标签：当前类别为1，其他类别为-1
    y_train_binary = np.where(y_train_encoded == idx, 1, -1)  # 105x1
    y_test_binary = np.where(y_test_encoded == idx, 1, -1)  # 45x1

    # 初始化SVM模型（linear，C=1e5模拟硬间隔）
    clf = SVC(kernel='linear', C=1e5)
    clf.fit(X_train, y_train_binary)
    
    # 存储模型参数
    classifiers[cls] = clf
    w_dict[cls] = clf.coef_[0].tolist()
    b_dict[cls] = clf.intercept_[0]
    support_vectors_dict[cls] = clf.support_.tolist()


# 为每个样本记录每个分类器的预测得分
train_predictions = np.zeros((X_train.shape[0], len(classes)))  # 训练集得分
test_predictions = np.zeros((X_test.shape[0], len(classes)))    # 测试集得分

for idx, cls in enumerate(classes):
    clf = classifiers[cls]
    train_predictions[:, idx] = clf.decision_function(X_train)
    test_predictions[:, idx] = clf.decision_function(X_test)

# 对每个样本选择得分最高的类作为预测结果
train_pred_final = le.inverse_transform(np.argmax(train_predictions, axis=1))  # axis=1: 按行
test_pred_final = le.inverse_transform(np.argmax(test_predictions, axis=1))

# 计算总错误率
total_train_error = np.sum(train_pred_final != y_train) / len(y_train)
total_test_error = np.sum(test_pred_final != y_test) / len(y_test)

# 计算每个小组的错误率
def calculate_group_errors(y_true, y_pred, classes):
    group_errors = {}
    for cls in classes:
        # 获取属于当前类别的样本
        cls_mask = (y_true == cls)
        total = np.sum(cls_mask)
        # 计算错误率
        errors = np.sum(y_pred[cls_mask] != cls)
        error_rate = errors / total
        group_errors[cls] = error_rate
    return group_errors

group_train_errors = calculate_group_errors(y_train, train_pred_final, classes)
group_test_errors = calculate_group_errors(y_test, test_pred_final, classes)

# 确定哪些类别在训练集上是线性可分的（训练错误为0）
linear_separable = [cls for cls in classes if group_train_errors[cls] == 0]

# 打印结果
print("\nQ2.2.2 Calculation using Standard SVM Model:")
print(f"total training error: {total_train_error}, total testing error: {total_test_error},")
print()

for cls in classes:
    print(f"class {cls}:")
    print(f"training error: {group_train_errors[cls]}, testing error: {group_test_errors[cls]},")
    print(f"w: [{', '.join(map(str, w_dict[cls]))}], b: {b_dict[cls]},")
    print(f"support vector indices: [{', '.join(map(str, support_vectors_dict[cls]))}],")
    # 输出空一行
    print()

print(f"Linear separable classes: {', '.join(linear_separable)}")


In [ ]:
# 定义C的值
C_values = [0.25 * t for t in range(1, 5)]

print('\nQ2.2.3 Calculation using SVM with Slack Variables (C=0.25×t, where t=1,...,4):')

for C in C_values:
    print("\n-------------------------------------------")
    print(f"C={C},")
    
    # 生成字典存储每个class的模型、weight、b、支持向量索引和错误率
    classifiers = {}
    w_dict = {}
    b_dict = {}
    support_vectors_dict = {}
    class_slack_vars = {}
    
    # 训练每个类别的SVM模型
    for idx, cls in enumerate(classes):
        # 创建二分类标签：当前类别为1，其他类别为-1
        y_train_binary = np.where(y_train_encoded == idx, 1, -1)
        y_test_binary = np.where(y_test_encoded == idx, 1, -1)
        
        # 初始化SVM模型（线性核，C为当前值）
        clf = SVC(kernel='linear', C=C)
        clf.fit(X_train, y_train_binary)
        
        # 存储模型参数
        classifiers[cls] = clf
        w_dict[cls] = clf.coef_[0].tolist()
        b_dict[cls] = clf.intercept_[0]
        support_vectors_dict[cls] = clf.support_.tolist()

        # 计算支持向量的松弛变量ζ
        support_indices = clf.support_
        support_vectors = X_train[support_indices]
        support_labels = y_train_binary[support_indices]
        decision_values = clf.decision_function(support_vectors)
        slack_vars = np.maximum(0, 1 - support_labels * decision_values)
        class_slack_vars[cls] = slack_vars.tolist()
    
    # 为每个样本记录每个分类器的预测得分
    train_predictions = np.zeros((X_train.shape[0], len(classes)))  # 训练集得分
    test_predictions = np.zeros((X_test.shape[0], len(classes)))    # 测试集得分
    
    for idx, cls in enumerate(classes):
        clf = classifiers[cls]
        train_predictions[:, idx] = clf.decision_function(X_train)
        test_predictions[:, idx] = clf.decision_function(X_test)
    
    # 对每个样本选择得分最高的类作为预测结果
    train_pred_final = le.inverse_transform(np.argmax(train_predictions, axis=1))  # axis=1: 按行
    test_pred_final = le.inverse_transform(np.argmax(test_predictions, axis=1))
    
    # 计算总错误率
    total_train_error = np.sum(train_pred_final != y_train) / len(y_train)
    total_test_error = np.sum(test_pred_final != y_test) / len(y_test)
    
    # 计算每个小组的错误率
    group_train_errors = calculate_group_errors(y_train, train_pred_final, classes)
    group_test_errors = calculate_group_errors(y_test, test_pred_final, classes)
    
    print(f"total training error: {total_train_error}, total testing error: {total_test_error},")
    print()
    
    for cls in classes:
        print(f"class {cls}:")
        print(f"training error: {group_train_errors[cls]}, testing error: {group_test_errors[cls]},")
        print(f"w: [{', '.join(map(str, w_dict[cls]))}], b: {b_dict[cls]},")
        print(f"support vector indices: [{', '.join(map(str, support_vectors_dict[cls]))}],")
        print(f"slack variable: {class_slack_vars[cls]},")
        print()
    


In [ ]:
# 定义 kernel 类型
kernel_types = [
    ('2nd-order Polynomial Kernel', 'poly', 2, 'scale'),
    ('3rd-order Polynomial Kernel', 'poly', 3, 'scale'),
    ('Radial Basis Function Kernel with σ = 1', 'rbf', None, 0.5),
    ('Sigmoidal Kernel with σ = 1', 'sigmoid', None, 0.5)
]

print("\nQ2.2.4 Calculation using SVM with Kernel Functions:")

for i, (description, kernel, degree, gamma) in enumerate(kernel_types, start=1):
    print("-------------------------------------------")
    print(f"({chr(96 + i)}) {description},")
    
    # 生成字典存储每个class的模型、weight、b、支持向量索引
    classifiers = {}
    w_dict = {}
    b_dict = {}
    support_vectors_dict = {}
    
    # 训练每个类别的SVM模型
    for idx, cls in enumerate(classes):
        # 创建二分类标签：当前类别为1，其他类别为-1
        y_train_binary = np.where(y_train_encoded == idx, 1, -1)
        y_test_binary = np.where(y_test_encoded == idx, 1, -1)
        
        # 初始化SVM模型
        if kernel == 'poly':
            clf = SVC(kernel=kernel, degree=degree, gamma=gamma, C=1e5)
        elif kernel == 'rbf':
            clf = SVC(kernel=kernel, gamma=gamma, C=1e5)
        elif kernel == 'sigmoid':
            clf = SVC(kernel=kernel, gamma=1, C=1e5)  # Here we equate gamma to sigma (1)
        else:  # 应对特殊情况
            clf = SVC(kernel=kernel, C=1e5)
        
        # 训练SVM模型
        clf.fit(X_train, y_train_binary)
        
        # 存储模型参数
        classifiers[cls] = clf
        if kernel == 'linear':
            w = clf.coef_[0].tolist()
        else:
            w = ' '  # 非线性核的w设为' '
        w_dict[cls] = w
        b_dict[cls] = clf.intercept_[0]
        support_vectors_dict[cls] = clf.support_.tolist()
    
    # 为每个样本记录每个分类器的预测得分
    train_predictions = np.zeros((X_train.shape[0], len(classes)))  # 训练集得分
    test_predictions = np.zeros((X_test.shape[0], len(classes)))    # 测试集得分
    
    for idx, cls in enumerate(classes):
        clf = classifiers[cls]
        train_predictions[:, idx] = clf.decision_function(X_train)
        test_predictions[:, idx] = clf.decision_function(X_test)
    
    # 对每个样本选择得分最高的类作为预测结果
    train_pred_final = le.inverse_transform(np.argmax(train_predictions, axis=1))  # axis=1: 按行
    test_pred_final = le.inverse_transform(np.argmax(test_predictions, axis=1))
    
    # 计算总错误率
    total_train_error = np.sum(train_pred_final != y_train) / len(y_train)
    total_test_error = np.sum(test_pred_final != y_test) / len(y_test)
    
    # 计算每个小组的错误率
    group_train_errors = calculate_group_errors(y_train, train_pred_final, classes)
    group_test_errors = calculate_group_errors(y_test, test_pred_final, classes)
    
    print(f"total training error: {total_train_error}, total testing error: {total_test_error},")
    print()
    
    for cls in classes:
        print(f"class {cls}:")
        print(f"training error: {group_train_errors[cls]}, testing error: {group_test_errors[cls]},")
        print(f"w: {w_dict[cls]}, b: {b_dict[cls]},")
        print(f"support vector indices: {support_vectors_dict[cls]},")
        print()
    
